In [1]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [2]:
model = ChatOpenAI(temperature=0)
# Define your desired data structure.
from pydantic import BaseModel, Field
from enum import Enum

class EmailType(str, Enum):
    spam = "spam"
    reminder = "reminder"
    others = "others"

class EmailParser(BaseModel):
    email_sender: str = Field(
        description="The email address of the sender, indicating who sent the email."
    )
    email_receiver: str = Field(
        description="The email address of the recipient, indicating who is intended to receive the email."
    )
    email_body: str = Field(
        description="The main content of the email, which may include text, links, or other information. This field represents the message being conveyed from the sender to the receiver."
    )
    email_type: EmailType = Field(
        description="The classification of the email based on its content or purpose. It can be one of the following: "
                    "'spam' for unsolicited or irrelevant emails, "
                    "'reminder' for messages that serve as a reminder or notification, "
                    "'others' for messages that serve other than spam and reminder "
    )

model = ChatOpenAI(temperature=0)


In [3]:
email = """
Dear Jacob,  
  
Your opinion matters to us actually and we love you so much!!!!  
  
You are invited to participate in a short survey about local transportation
issues in the Bay Area.  Your opinions will be used for research purposes only
and your responses will remain completely confidential.  
  
Follow this link to the Survey:  
Take the Survey  
  
Or copy and paste the URL below into your internet browser:  
https://thepollingplace.qualtrics.com/jfe/form/SV_aibZ5geNkxuBng2?Q_DL=4BQDYnCmFixO9bW_aibZ5geNkxuBng2_CGC_wYZc0ZnW5tL60xa&Q_CHL=email  
  
Thank you for your participation!  
  
EMC Research is a national market and opinion research firm that conducts
research on a variety of topics. We are not trying to sell anything, and your
responses will remain completely anonymous. To learn more about EMC Research,
go to www.emcresearch.com.  
  
Follow the link to opt out of future emails:  
Click here to unsubscribe


-----------

Dear Jacob,  
  
Your opinion matters to us!  
  
You are invited to participate in a short survey about local transportation
issues in the Bay Area.  Your opinions will be used for research purposes only
and your responses will remain completely confidential.  
  
Follow this link to the Survey:  
Take the Survey  
  
Or copy and paste the URL below into your internet browser:  
https://thepollingplace.qualtrics.com/jfe/form/SV_aibZ5geNkxuBng2?Q_DL=4BQDYnCmFixO9bW_aibZ5geNkxuBng2_CGC_wYZc0ZnW5tL60xa&Q_CHL=email  
  
Thank you for your participation!  
  
EMC Research is a national market and opinion research firm that conducts
research on a variety of topics. We are not trying to sell anything, and your
responses will remain completely anonymous. To learn more about EMC Research,
go to www.emcresearch.com.  
  
Follow the link to opt out of future emails:  
Click here to unsubscribe


Bamboo Spa Grand Reopening Sale!

|  |  
---  
|

4 Options Starting From

$70.00  
  
  
---  
  

View Offer  
  
What We're Offering

**Join in our reopening celebration today by enjoying exclusive offers on
your preferred services!**

Unwind and rejuvenate yourself with our exclusive packages of **Foot
Reflexology** , **Tension Tamer** , and **CustomBlend Massage**.

Make sure to visit us for our inaugural=C2=A0 **Halloween celebration on
October 27th, from 3pm-6pm </strong> (admission is free), where you can
indulge in delightful treats and take advantage of amazing deals **RSVP**
**.** For further details aboutthis month's promotions, kindly refer to our
**=C2=A0** **Specials Page.

**Special Offers for the Grand Reopening!**

****__* 1 hr Foot Reflexology $70__

____* 30 min Tension Tamer Massage + 30 min Foot Reflexology$85__

____* 60 min Custom Blend Massage $105__

____* 90 min Custom Blend Massage $125__

**Foot Reflexology:=C2=A0** Revitalizing and refreshing the feet and legs is
accomplished by stimulating key pressure points either manually or using a
reflexology stick, through foot reflexology.

**Tension Tamer:=C2=A0** Ease tension in your neck or unwindwith a soothing
scalp massage. Upon request, enjoy essential oils or lotion; otherwise
receive a dry treatment.

**Custom Blend Massage:** =C2=A0Our expert therapists with awealth of
experience will skillfully combine various techniques to relievetension,
promote relaxation and provide relief for your muscles.

For questions or more information, please call 415-567-8812 or visit our=
website at http://www.bamboospasf.com.  
  
|  

Bamboo Spa SF

415-567-8812

www.bamboospasf.com

frontdesk@bamboospasf.com

|

|

2284 Union St  
San Francisco, CA 94123  
  
---  
  
|  Find more local experiences and promotions  
|

  

|  
  
---|---  
This email was created by Hownd on behalf of Bamboo Spa SF  
(Located at 2284 Union St, San Francisco, CA, 94123).  
  
  
Please review our Privacy Policy.  
Unsubscribe to stop receiving this email.  
Manage Privacy
"""

In [4]:
import requests
import json

def send_request():
    url = "http://localhost:11434/api/generate"
    data = {
        "model": "minicpm-v:8b",
        "prompt": "Why is the sky blue?",
        "stream": False,
        "options": {
            "temperature": 0
        }
    }
    
    headers = {'Content-Type': 'application/json'}
    
    response = requests.post(url, data=json.dumps(data), headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

response_data = send_request()
print(response_data['response'])


The color of the sky appears to be a shade of blue because it reflects sunlight. The Earth's atmosphere scatters light in all directions, but short-wavelength visible light (blue and violet) are scattered more than others due to their shorter wavelengths. This scattering effect causes most of this light to scatter into our eyes from every direction we look at the sky during daylight hours.

The reason why blue is a dominant color for sunlight that reaches us on Earth's surface, as opposed to other colors like red or green, has to do with how much each wavelength scatters in different directions. Blue and violet wavelengths are scattered more than others because they have shorter wavelengths which means their particles move faster through the air molecules before hitting another one.

The scattering of light is also why we see stars at night as bright points instead of diffuse disks like during daytime when all visible light from a star has been scattered in every direction by Earth's a

In [10]:
from typing import Any, Dict, Iterator, List, Mapping, Optional
import requests
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk

# https://python.langchain.com/docs/how_to/custom_llm/ Tuto here
class CustomLLM(LLM):
    url: str
    is_stream: bool
    model: str
    model_path: str
    temperature: float = 0.7

    # def __init__(self, url: str, is_stream: bool, model: str, model_path: str):
    #     self.url = url
    #     self.is_stream = is_stream
    #     self.model = model
    #     self.model_path = model_path

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        print (self.temperature)
        data = {
            "model": self.model,
            "prompt": prompt,
            "stream": self.is_stream,
            "options": {
                "temperature": self.temperature
            }
        }
        
        # Make the POST request
        headers = {'Content-Type': 'application/json'}
        response = requests.post(url=self.url, json=data, headers=headers)
        
        if response.status_code == 200:
            response_data = response.json()
            # Assuming the API response contains the generated text in a field like 'output'
            return response_data.get("response") # Return n characters from the output
        else:
            raise RuntimeError(f"Error {response.status_code}: {response.text}")


    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"

In [11]:
llm = CustomLLM(url = "http://localhost:11434/api/generate",is_stream = False,  model = "minicpm-v:8b", model_path="minicpm-v:8b")

In [17]:
parser = JsonOutputParser(pydantic_object=EmailParser)

prompt = PromptTemplate(
    template="Can you provide details or insights on the very first email found at the top of the document below? \nMake sure to extract the email in the following format.{format_instructions}.\nHere is the document:\n{input}",
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser


llm.temperature = 1.0
chain.invoke({"input": email})

1.0


{'email_sender': 'unspecified',
 'email_receiver': 'Jacob',
 'email_body': [{'text': 'Dear Jacob,\n\nYour opinion matters to us actually and we love you so much!!\nYou are invited to participate in a short survey about local transportation issues in the Bay Area. Your opinions will be used for research purposes only and your responses will remain completely confidential.\n\nFollow this link to the Survey:\nTake the Survey \nor copy and paste the URL below into your internet browser: https://thepollingplace.qualtrics.com/jfe/form/SV_aibZ5geNkxuBng2?Q_DL=4BQDYnCmFixO9bW_aibZ5geNkxuBng2_CGC_wYZc0ZnW5tL60xa&Q_CHL=email\nThank you for your participation!\nEMC Research is a national market and opinion research firm that conducts research on a variety of topics. We are not trying to sell anything, and your responses will remain completely anonymous.\nTo learn more about EMC Research, go to www.emcresearch.com.\nFollow the link to opt out of future emails:\nClick here to unsubscribe'},
  {'tex

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"EmailType": {"enum": ["spam", "reminder", "others"], "title": "EmailType", "type": "string"}}, "properties": {"email_sender": {"description": "The email address of the sender, indicating who sent the email.", "title": "Email Sender", "type": "string"}, "email_receiver": {"description": "The email address of the recipient, indicating who is intended to receive the email.", "title": "Email Receiver", "type": "string"}, "email_body": {"description": "The main content of the email, which may include text, links, or other information. T